![](files/Fasshauer.png)

In [ ]:
using SpecialFunctions, Laplacians, KernelMatrices, LinearAlgebra, Plots

function dist(x, z, d = 2)
    return (sum((x .- z).^d))^(1/d)
end

function matkern(dis::Number, epsilon::Number, m::Number, d::Int = 2)
    if dis == 0.0 
        return 1.0
    else
        return besselk(m-d/2, epsilon*dis)*(epsilon*dis)^(m-d/2)
    end
end

┌ Info: Precompiling Laplacians [6f8e5838-0efe-5de0-80a3-5fb4f8dbb1de]
└ @ Base loading.jl:1278


In [ ]:
L = 32
K = 16
a_num = (L,K) # length vectors for a 2D problem
x1 = 1:a_num[1]
x2 = 1:a_num[2]
d = length(a_num)
img = sin.(2*pi*x1/16)*sin.(2*pi*x2'/8) + randn(a_num)*0.1


cent = [(7,6), (15,12), (22,10), (26, 8)]
rad = 2.0

function prod_dot(m)
    mas = m[1]
    for i = 2:length(m)
        mas .*= m[i]
    end
    return mas
end

mask = map(c->broadcast(+, (x1 .- c[1]).^2, (x2 .- c[2])'.^2) .> rad^2, cent)
mask = prod_dot(mask)

# Find where the data are missing
inds = sortperm(mask[:])
S = L*K-sum(mask)
missing_inds = inds[1:S]
println(missing_inds)
nonmissing_inds = inds[(S+1):end]

println("There are $S missing.")

p1 = heatmap(img, title = "image")
p2 = heatmap(mask, title = "holes")

plot(p1,p2,layout=(1,2))

From Rasmussen and Williams book:
![](files/RW_GP_Pred.png)

In [ ]:
# No noise is assumed, sigsq_n = 0

xx = [[x,y] for x in x1 for y in x2]
xxyy = [(x,y) for x in xx[inds] for y in xx[inds]]
dis = reshape(map(x -> dist(x...), xxyy), (512, 512))

epsilon = 0.22
m = 2
M = matkern.(dis, epsilon, m)

chK = cholesky(M[nonmissing_inds, nonmissing_inds])
alpha = chK.L'\(chK.L\img[nonmissing_inds])
fbar = M[nonmissing_inds,missing_inds]'*alpha

In [ ]:
img_int = copy(img)
img_int[missing_inds] = fbar

p3 = heatmap(img_int, title="interpolation")

plot(p1,p3,p2,layout=(1,3),size=(1000,300))

In [ ]:
hcat(img[missing_inds], fbar)